In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import h5py
from matplotlib import pyplot as plt
import torch
import numpy as np
from pytorch3dunet.unet3d.losses import *
from pytorch3dunet.unet3d.metrics import MeanIoU
from pathlib import Path
import glob
import os

import matplotlib
#matplotlib.use('TkAgg')

%matplotlib inline

In [2]:
class DiceProbLoss(DiceLoss):
    def __init__(self):
        super(DiceProbLoss, self).__init__(normalization='none')

class MeanIoUBin(MeanIoU):
    def __init__(self):
        super(MeanIoUBin, self).__init__(is_binarized=True)

class BCEDiceBinLoss(nn.Module):
    """Linear combination of BCE and Dice losses"""

    def __init__(self, alpha, beta):
        super(BCEDiceBinLoss, self).__init__()
        self.alpha = alpha
        self.bce = nn.BCELoss()
        self.beta = beta
        self.dice = DiceBinLoss()

    def forward(self, input, target):
        return self.alpha * self.bce(input, target) + self.beta * self.dice(input, target)


initLosses = {
    "BCE": nn.BCELoss,
    "Dice": DiceProbLoss,
    "MeanIoU": MeanIoU
}

In [25]:
basepred = Path('/home/lorenzo/3dunet-cavity/runs/run_210623_gpu/predictions')
baseorig = Path('/home/lorenzo/deep_apbs/destData/refined-set_filter')

def genDataSets():

    for predfname in glob.glob(str(basepred / '*_grids_predictions.h5')):
        name = Path(predfname).name.split('_')[0]
        labelfname = baseorig / name / f'{name}_grids.h5'
        
        if not os.path.exists(labelfname):
            print(f'{labelfname} does not exist.')
            continue
        
        labelT = torch.tensor(h5py.File(labelfname)['label'], dtype=torch.float32)
        labelT = labelT[None,None]
        predT = torch.tensor(h5py.File(predfname)['predictions'])
        predT = predT[None]
        
        yield name, (predT, labelT)

In [26]:
class RunningAverage:
    def __init__(self, loss):
        self.count = 0
        self.sum = 0
        self.loss = loss
        self.losses = {}

    def update(self, pred, label, name):
        self.count += 1
        loss = self.loss(pred, label).item()
        self.sum += loss
        self.losses[name] = loss

    def value(self):
        return self.sum / self.count
    
class AverageLosses:
    def __init__(self, losses, kwargs=None):
        if kwargs is None:
            kwargs = {name:{} for name in losses.keys()}
        self.losses = {name: RunningAverage(loss(**kwargs[name])) for name,loss in losses.items()}

    def update(self, pred, label, prot):
        for name in self.losses.keys():
            self.losses[name].update(pred,label, prot)
        
    def value(self):
        return {name: loss.value() for name,loss in self.losses.items()}

In [27]:
OracleLoss = AverageLosses(initLosses)
UnetLoss = AverageLosses(initLosses)
RandLoss = AverageLosses(initLosses)
RandUnitLoss = AverageLosses(initLosses)
ZeroLoss = AverageLosses(initLosses)
UnitLoss = AverageLosses(initLosses)

for name, (predT, labelT) in genDataSets():
    constPred = torch.zeros_like(labelT)
    ZeroLoss.update(constPred, labelT, name)

    constPred[:] = 1
    UnitLoss.update(constPred, labelT, name)

    UnetLoss.update(predT, labelT, name)

    OracleLoss.update(labelT, labelT, name)

    randPred = torch.rand(size=constPred.size(), dtype=constPred.dtype, device=constPred.device, requires_grad=False)
    RandLoss.update(randPred, labelT, name)
    
    randPred[randPred < 0.5] = 0
    randPred[randPred > 0.5] = 1
    RandUnitLoss.update(randPred, labelT, name)

/home/lorenzo/deep_apbs/destData/refined-set_filter/1oba/1oba_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/2epn/2epn_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1tkb/1tkb_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1rnt/1rnt_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/2h21/2h21_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/2am4/2am4_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1x8d/1x8d_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/2euk/2euk_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1lnm/1lnm_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1o1s/1o1s_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1r9l/1r9l_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-se

In [28]:
RandUnitLoss.value(), RandLoss.value(), UnitLoss.value(), ZeroLoss.value(),OracleLoss.value()

({'BCE': 49.99950144841121,
  'Dice': 0.997966271180373,
  'MeanIoU': 0.0010180229527087738},
 {'BCE': 0.9999702343573937,
  'Dice': 0.9969496108018435,
  'MeanIoU': 0.0010180229527087738},
 {'BCE': 99.89769275371845,
  'Dice': 0.997956259892537,
  'MeanIoU': 0.001023037529263932},
 {'BCE': 0.1023037525323721, 'Dice': 1.0, 'MeanIoU': 0.0},
 {'BCE': 0.0, 'Dice': 0.0, 'MeanIoU': 1.0})

In [32]:
UnetLoss.losses['MeanIoU'].losses

{'1sw2': 0.4645071029663086,
 '1fiv': 0.6156509518623352,
 '1a4r': 0.17319780588150024,
 '4gbd': 0.6333988904953003,
 '5g1z': 0.12134632468223572,
 '5gja': 0.3489736020565033,
 '2fqt': 0.29123446345329285,
 '4r75': 0.567950963973999,
 '4non': 0.15870153903961182,
 '3fjg': 0.5126749873161316,
 '3s0e': 0.493194043636322,
 '1dy4': 0.5372849106788635,
 '1dqn': 0.2584525942802429,
 '1m48': 0.027618248015642166,
 '5yz2': 0.20882698893547058,
 '4fxp': 0.48952972888946533,
 '5mxf': 0.0,
 '1ctt': 0.0,
 '1ceb': 0.2004687339067459,
 '5km9': 0.18358531594276428,
 '2dw7': 0.008737157098948956,
 '1t7d': 0.22022944688796997,
 '5tcj': 0.03971793130040169,
 '5vij': 0.4260115623474121,
 '4jwk': 0.23512376844882965,
 '5tuz': 0.09520634263753891}

In [ ]:
thress = np.linspace(0.1,1.0,10)

kwargs = {f"MeanIoU_{thres}": {'thres':thres} for thres in thress}
initLosses = {f"MeanIoU_{thres}": MeanIoU for thres in thress}

UnetLosses = AverageLosses(initLosses, kwargs)

for name, (predT, labelT) in genDataSets():
    UnetLosses.update(predT, labelT, name)

In [ ]:
UnetLosses.losses['MeanIoU'].losses

## Pdb scores

In [3]:
import h5py
import numpy as np
import prody
import prody as pr
import os
from potsim2 import PotGrid
import sklearn
from sklearn.metrics import matthews_corrcoef as mcc 
from sklearn.metrics import f1_score

predbase = f'/home/lorenzo/3dunet-cavity/runs/run_210720_gpu'
predbaseKala = '/home/lorenzo/kalasanty/predictions_210723'

datadir = '/home/lorenzo/deep_apbs/destData/refined-set_filter_1'

In [4]:
def yieldDataTheirs():

    for fname in os.listdir(predbaseKala):
        name = fname.split('_')[0]

        predfname = f'{predbaseKala}/{name}_predictions.pdb'
        pdbfname = f'{datadir}/{name}/{name}_selected.pdb'
        pocketfname = f'{datadir}/{name}/{name}_pocket.pdb'

        structure = prody.parsePDB(pdbfname)
        pred = prody.parsePDB(predfname)
        pocket = prody.parsePDB(pocketfname)
        
        yield name, (structure, pred, pocket)

def yieldDataMine():

    for fname in os.listdir(f'{predbase}/predictions'):
        name = fname.split('_')[0]

        predfname = f'{predbase}/predictions/{name}_predictions.pdb'
        pdbfname = f'{datadir}/{name}/{name}_selected.pdb'
        pocketfname = f'{datadir}/{name}/{name}_pocket.pdb'

        structure = prody.parsePDB(pdbfname)
        pred = prody.parsePDB(predfname)
        pocket = prody.parsePDB(pocketfname)
        
        yield name, (structure, pred, pocket)

        
def calcMetric(pred, pocket, structure, scoref):
    prednums = set(t.getResnum() for t in pred.iterResidues())
    truenums = set(t.getResnum() for t in pocket.iterResidues())
    allnums = set(t.getResnum() for t in structure.iterResidues())

    tp = 0
    tn = 0
    fp = 0
    fn = 0

    y_true = []
    y_pred = []

    for num in allnums:
        if num not in prednums and num not in truenums:
            tn += 1
            y_pred.append(-1)
            y_true.append(-1)
        elif num not in prednums and num in truenums:
            fn += 1
            y_pred.append(-1)
            y_true.append(1)
        elif num in prednums and num not in truenums:
            fp += 1
            y_pred.append(1)
            y_true.append(-1)
        elif num in prednums and num in truenums:
            tp += 1
            y_pred.append(1)
            y_true.append(1)
        else:
            raise Exception

    return scoref(y_true, y_pred)

In [5]:
def scoresFromGen(gen):
    names = [] 
    mcc_scores = []
    f_scores = []
    for name, (structure, pred, pocket) in gen():
        yield name, (calcMetric(pred, pocket, structure, f1_score), calcMetric(pred, pocket, structure, mcc))

In [6]:
resTheirs = list(scoresFromGen(yieldDataTheirs))

In [7]:
resOurs = list(scoresFromGen(yieldDataMine))

In [8]:
np.array([list(r[1]) for r in resTheirs]).mean(axis=0)

array([0.47458644, 0.4496932 ])

In [9]:
np.array([list(r[1]) for r in resOurs]).mean(axis=0)

array([0.39136112, 0.37283942])

In [10]:
def mergeDict(d1, d2):
    keys = set(d1.keys()) | set(d2.keys())
    return {key:(d1.get(key),d2.get(key)) for key in keys}

In [11]:
dictOurs = {x[0]: x[1][0] for x in resOurs}
dictTheirs = {x[0]: x[1][0] for x in resTheirs}
d2 = mergeDict(dictOurs, dictTheirs)

d2

{'5wxh': (0.3333333333333333, None),
 '3b4p': (0.6060606060606061, 0.10000000000000002),
 '4pzv': (0.2156862745098039, 0.0),
 '5kly': (0.7333333333333334, 0.7619047619047621),
 '3hmp': (0.3466666666666666, 0.4680851063829787),
 '3wto': (0.22222222222222224, 0.14285714285714285),
 '5t7s': (0.23333333333333336, 0.5925925925925927),
 '1m48': (0.0, 0.08333333333333333),
 '5fnu': (0.65625, 0.5),
 '5vij': (0.46341463414634143, 0.7894736842105263),
 '1ctt': (0.6216216216216216, 0.0),
 '2gsu': (0.45945945945945943, 0.634920634920635),
 '2zx7': (0.56, 0.7346938775510204),
 '4rd6': (0.0, None),
 '2vzr': (0.4444444444444445, 0.7407407407407407),
 '2vc9': (0.5633802816901409, 0.0),
 '3t1a': (0.0, None),
 '2dw7': (0.13793103448275865, 0.06060606060606061),
 '1oba': (0.0, 0.0),
 '4bcn': (0.5217391304347826, 0.7499999999999999),
 '2d0k': (0.625, 0.6),
 '2gyi': (0.45714285714285713, 0.588235294117647),
 '3rm9': (0.47761194029850745, 0.48484848484848486),
 '3o9p': (0.6444444444444445, 0.673684210526315

In [13]:
d3 = {k : v for k,v in d2.items() if v[0] is not None and v[1] is not None}
d4 = np.array([list(v) for v in d3.values()])

d4.mean(axis=0)

array([0.40322958, 0.47458644])

In [27]:
d4[:,0].std()

0.1839451691877432

In [28]:
d4[:,1].std()

0.31767485863589817

In [24]:
(d4[:,0]-d4[:,1]).mean(), (d4[:,0]-d4[:,1]).std()

(-0.07135685487063302, 0.3125327462276734)

In [14]:
d3

{'3b4p': (0.6060606060606061, 0.10000000000000002),
 '4pzv': (0.2156862745098039, 0.0),
 '5kly': (0.7333333333333334, 0.7619047619047621),
 '3hmp': (0.3466666666666666, 0.4680851063829787),
 '3wto': (0.22222222222222224, 0.14285714285714285),
 '5t7s': (0.23333333333333336, 0.5925925925925927),
 '1m48': (0.0, 0.08333333333333333),
 '5fnu': (0.65625, 0.5),
 '5vij': (0.46341463414634143, 0.7894736842105263),
 '1ctt': (0.6216216216216216, 0.0),
 '2gsu': (0.45945945945945943, 0.634920634920635),
 '2zx7': (0.56, 0.7346938775510204),
 '2vzr': (0.4444444444444445, 0.7407407407407407),
 '2vc9': (0.5633802816901409, 0.0),
 '2dw7': (0.13793103448275865, 0.06060606060606061),
 '1oba': (0.0, 0.0),
 '4bcn': (0.5217391304347826, 0.7499999999999999),
 '2d0k': (0.625, 0.6),
 '2gyi': (0.45714285714285713, 0.588235294117647),
 '3rm9': (0.47761194029850745, 0.48484848484848486),
 '3o9p': (0.6444444444444445, 0.6736842105263158),
 '5vi6': (0.08, 0.19354838709677416),
 '1a4r': (0.0, 0.6666666666666667),
 '4

In [75]:
d5 = np.array([v[0] > v[1] for v in d4])
d5.sum()/len(d5)

0.30456852791878175

In [78]:
(d4 == 0).sum(axis=0)

array([ 8, 40])

In [13]:
mcc_scores = []
f_scores = []
for name, (structure, pred, pocket) in yieldDataTheirs():
    f_scores.append(calcMetric(pred, pocket, structure, f1_score))
    mcc_scores.append(calcMetric(pred, pocket, structure, mcc))

mcc_scores = np.array(mcc_scores)
f_scores = np.array(f_scores)

f_scores.mean(), mcc_scores.mean()

(0.4745864352032055, 0.4496931992768822)

## Scratch

In [33]:
import yaml
import h5py

conf = yaml.safe_load(open(rf'/home/lorenzo/3dunet-cavity/runs/210805_pdb/run_config.yml','r'))

In [ ]:
h5py.File()

In [32]:
len(conf['val']) * (arr.size * 8) / 10**6

6977.725832

In [12]:
import numpy as np
import sys

arr = np.random.random(size=(161,161,161))

In [18]:
arr.dtype

dtype('float64')

33.386248

In [21]:
314 - 256

58

In [15]:
sys.getsizeof(arr)

33386376

In [2]:
import h5py
import numpy as np
import prody
import os
from potsim2 import PotGrid

predbase = f'/home/lorenzo/3dunet-cavity/runs/run_210628_gpu'

datadir = '/home/lorenzo/deep_apbs/destData/refined-set_filter'
name = '1a4r'

predfname = f'{predbase}/predictions/{name}_grids_predictions.h5'
pdbfname = f'{datadir}/{name}/{name}_selected.pdb'
ligandfname = f'{datadir}/{name}/{name}_ligand.pdb'

grids = h5py.File(predfname)
pred = np.array(grids['predictions'][0])
predbin = pred  > 0.5

structure = prody.parsePDB(pdbfname)
grid = PotGrid(f"{datadir}/{name}/{name}_selected.pdb", f"{datadir}/{name}/{name}_grid.dx.gz")

ligand = prody.parsePDB(ligandfname)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/lorenzo/3dunet-cavity/runs/run_210628_gpu/predictions/1a4r_grids_predictions.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
complx = ligand + structure
lresname = ligand.getResnames()[0]
pocket = complx.select(f'same residue as exwithin 4.5 of resname {lresname}')
prody.writePDB(f'{datadir}/{name}/{name}_pocket.pdb', pocket)
pocket

<Selection: 'same residue as... of resname GNH' from 1a4r_ligand + 1a4r_selected (246 atoms)>

In [5]:
print(min(grid.edges[0]), max(grid.edges[0]))
print(min(grid.edges[1]), max(grid.edges[1]))
print(min(grid.edges[2]), max(grid.edges[2]))

8.8395 169.8395
-53.3765 107.6235
-86.7575 74.2425


In [6]:
print(min(x[0] for x in structure.getCoords()), max(x[0] for x in structure.getCoords()))
print(min(x[1] for x in structure.getCoords()), max(x[1] for x in structure.getCoords()))
print(min(x[2] for x in structure.getCoords()), max(x[2] for x in structure.getCoords()))

60.64 118.039
6.425 47.822
-27.466 14.951


In [71]:
coords = []

for i,coord in enumerate(structure.getCoords()):
    x,y,z = coord
    binx = int((x - min(grid.edges[0])) / grid.delta[0])
    biny = int((y - min(grid.edges[1])) / grid.delta[1])
    binz = int((z - min(grid.edges[2])) / grid.delta[2])
    
    try:
        if predbin[binx,biny,binz]:
            coords.append(i)
    except IndexError as e:
        print(x,  min(grid.edges[0]), max(grid.edges[0]), grid.delta[0])
        print(binx,biny,binz)
        raise e

In [73]:
s = structure[[coords]]
lresname = s.getResnames()[0]

In [90]:
len(prody.AtomGroup())

0

In [84]:
structure.copy()

<AtomGroup: 1a4r_selected (3004 atoms)>

In [64]:
fname1 = '/home/lorenzo/deep_apbs/destData/refined-set_filter_1/2ydw/2ydw_selected.pdb'
fname2 = '/home/lorenzo/deep_apbs/destData/refined-set_filter_1/2ydw/2ydw_grid.dx.gz'

grid2  = PotGrid(fname1, fname2)

In [58]:
idxstr = ' '.join(map(str,s.getIndices()))
s2 = structure.select(f'same residue as index {idxstr}')
s2

<Selection: 'same residue as... 2780 2781 2783' from 1a4r_selected (845 atoms)>

In [ ]:
pocket = structure.select(f'same residue as {lresname}')

In [79]:
grid.grid = predbin.astype(int)
grid.export(f'{predbase}/{name}_predicted.dx')

In [ ]:
prody.parsePDB(pdbfname)

In [64]:
structure

<AtomGroup: 1a4r_selected (3004 atoms)>

In [63]:
prody.AtomGroup(s)

<AtomGroup: Selection 'index 57 61 64 ...80 to 2781 2783' (0 atoms; no coordinates)>

In [54]:
len(structure.getCoords()), len(coords)

(3004, 519)